# imports

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

from glob import glob as glob
from pathlib import Path

from sklearn.model_selection import train_test_split
import tensorflow as tf

# Getting Images

In [3]:
processed_crackdir = Path("../data/processed/pro_crack_img")
noncrack_dir = Path("../data/external/Non-crack")

def generate_df(image_dir, label):
    filepaths = pd.Series(list(image_dir.glob(r'*.jpg')), name='Filepath').astype(str)
    labels = pd.Series(label, name='Label', index=filepaths.index)
    df = pd.concat([filepaths, labels], axis=1)
    return df

In [4]:
crack_df = generate_df(processed_crackdir, label="CRACK_IMAGE")
noncrack_df = generate_df(noncrack_dir,label="NONCRACK_IMAGE")
total_df = pd.concat([crack_df,noncrack_df],axis=0).sample(frac=1.0,random_state=1).reset_index(drop=True)

total_df

,Filepath,Label
0,..\data\external\Non-crack\08623.jpg,NONCRACK_IMAGE
1,..\data\external\Non-crack\05028.jpg,NONCRACK_IMAGE
2,..\data\external\Non-crack\15926.jpg,NONCRACK_IMAGE
3,..\data\external\Non-crack\15710.jpg,NONCRACK_IMAGE
4,..\data\external\Non-crack\17857.jpg,NONCRACK_IMAGE
...,...,...
20005,..\data\external\Non-crack\10946.jpg,NONCRACK_IMAGE
20006,..\data\external\Non-crack\17280.jpg,NONCRACK_IMAGE
20007,..\data\external\Non-crack\05183.jpg,NONCRACK_IMAGE
20008,..\data\external\Non-crack\12163.jpg,NONCRACK_IMAGE


# Test-Train split

In [5]:
train_df, test_df = train_test_split(
    total_df.sample(6000, random_state=1),
    train_size=0.7,
    shuffle=True,
    random_state=1
)

# Loading/Adding Images

In [6]:
train_gen = tf.keras.preprocessing.image.ImageDataGenerator(
                rescale = 1./255,
                validation_split = 0.2
)

test_gen = tf.keras.preprocessing.image.ImageDataGenerator(
                rescale = 1./255
)

In [7]:
train_data = train_gen.flow_from_dataframe(
    train_df,
    x_col='Filepath',
    y_col='Label',
    target_size=(120, 120),
    color_mode='rgb',
    class_mode='binary',
    batch_size=24,
    shuffle=True,
    seed=42,
    subset='training'
)

val_data = train_gen.flow_from_dataframe(
    train_df,
    x_col='Filepath',
    y_col='Label',
    target_size=(120, 120),
    color_mode='rgb',
    class_mode='binary',
    batch_size=24,
    shuffle=True,
    seed=42,
    subset='validation'
)

test_data = train_gen.flow_from_dataframe(
    test_df,
    x_col='Filepath',
    y_col='Label',
    target_size=(120, 120),
    color_mode='rgb',
    class_mode='binary',
    batch_size=24,
    shuffle=False,
    seed=42
)

Found 3360 validated image filenames belonging to 2 classes.
Found 840 validated image filenames belonging to 2 classes.
Found 1800 validated image filenames belonging to 2 classes.


# Training

In [ ]:
inputs = tf.keras.Input(shape=(120, 120, 3))
x = tf.keras.layers.Conv2D(filters=12, kernel_size=(3, 3), activation='relu')(inputs)
x = tf.keras.layers.MaxPool2D(pool_size=(2, 2))(x)
x = tf.keras.layers.Conv2D(filters=24, kernel_size=(3, 3), activation='relu')(x)
x = tf.keras.layers.MaxPool2D(pool_size=(2, 2))(x)
x = tf.keras.layers.GlobalAveragePooling2D()(x)
outputs = tf.keras.layers.Dense(1, activation='sigmoid')(x)

model = tf.keras.Model(inputs=inputs, outputs=outputs)

model.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics=['accuracy']
)

print(model.summary())

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, 120, 120, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d (Conv2D)                 │ (None, 118, 118, 12)   │           336 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 59, 59, 12)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 57, 57, 24)     │         2,616 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 28, 28, 24)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d        │ (None, 24)             │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 1)              │            25 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,977 (11.63 KB)

 Trainable params: 2,977 (11.63 KB)

 Non-trainable params: 0 (0.00 B)

None


In [9]:
history = model.fit(
    train_data,
    validation_data=val_data,
    epochs=100,
    callbacks=[
        tf.keras.callbacks.EarlyStopping(
            monitor='val_loss',
            patience=3,
            restore_best_weights=True
        )
    ]
)

c:\Users\saiprasath\ML-Surface-Crack-Detection\.venv\lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/100
140/140 ━━━━━━━━━━━━━━━━━━━━ 86s 594ms/step - accuracy: 0.9998 - loss: 0.1231 - val_accuracy: 0.9976 - val_loss: 0.0056
Epoch 2/100
140/140 ━━━━━━━━━━━━━━━━━━━━ 13s 94ms/step - accuracy: 1.0000 - loss: 5.7223e-04 - val_accuracy: 0.9976 - val_loss: 0.0056
Epoch 3/100
140/140 ━━━━━━━━━━━━━━━━━━━━ 20s 90ms/step - accuracy: 0.9996 - loss: 0.0011 - val_accuracy: 0.9976 - val_loss: 0.0053
Epoch 4/100
140/140 ━━━━━━━━━━━━━━━━━━━━ 13s 91ms/step - accuracy: 0.9999 - loss: 2.7091e-04 - val_accuracy: 0.9976 - val_loss: 0.0047
Epoch 5/100
140/140 ━━━━━━━━━━━━━━━━━━━━ 21s 92ms/step - accuracy: 0.9993 - loss: 0.0012 - val_accuracy: 0.9976 - val_loss: 0.0040
Epoch 6/100
140/140 ━━━━━━━━━━━━━━━━━━━━ 20s 90ms/step - accuracy: 0.9998 - loss: 3.6162e-04 - val_accuracy: 0.9976 - val_loss: 0.0032
Epoch 7/100
140/140 ━━━━━━━━━━━━━━━━━━━━ 13s 90ms/step - accuracy: 0.9994 - loss: 6.0103e-04 - val_accuracy: 0.9976 - val_loss: 0.0023
Epoch 8/100
140/140 ━━━━━━━━━━━━━━━━━━━━ 12s 86ms/step - accuracy:

KeyboardInterrupt: 

In [ ]:
print(history.history.keys())

fig = plt.line(
    history.history,
    y=['loss', 'val_loss'],
    labels={'index': "Epoch", 'value': "Loss"},
    title="Training and Validation Loss Over Time"
)
fig.show()